In [3]:
import pandas as pd
from dotenv import load_dotenv
from os import getenv
from sqlalchemy import create_engine

In [4]:
load_dotenv()

DB_CONFIG = {
    'user': getenv('DB_ROOT'),
    'password': getenv('DB_ROOT_PASSWORD'),
    'host': getenv('DB_HOST'),
    'database': getenv('DB_NAME'),
    'port': getenv('DB_PORT')
}

engine = create_engine(f"postgresql+psycopg2://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}")

In [67]:
average_age = pd.read_json('../analysis/average_age/average_age_per_team.json')
average_age = average_age.rename(columns={
    'TeamID': 'team_id',
    'Year': 'year',
    'AverageAge': 'average_age'
})

average_points = pd.read_json('../analysis/average_points/average_points_per_team_edited.json')
average_points = average_points.rename(columns={
    'TeamID': 'team_id',
    'Year': 'year',
    'Team_name': 'team_name',
    'AveragePoints': 'average_points'
})

club_info = pd.read_json('../analysis/club_info/club_info.json')
club_info = club_info.rename(columns={
    'TeamID': 'team_id',
    'Team_name': 'team_name',
    'NumberOfCups': 'number_of_cups',
    'NationalTeamID': 'national_team_id',
    'ImageLink': 'image_link'
})

club_titles = pd.read_json('../analysis/club_titles/club_titles.json')
club_titles = club_titles.rename(columns={
    'TeamID': 'team_id',
    'Year': 'year',
    'NumberOfTitlesThisYear': 'number_of_titles_this_year'
})

country_info = pd.read_json('../analysis/country_info/country_info.json')
country_info = country_info.rename(columns={
    'NationalTeamID': 'national_team_id',
    'NationalTeamName': 'national_team_name',
    'ClubIDs': 'team_ids'
})

legionnaires_per_team = pd.read_json('../analysis/legionnaires/legionnaires_per_team.json')
legionnaires_per_team = legionnaires_per_team.rename(columns={
    'TeamID': 'team_id',
    'Year': 'year',
    'Legioners': 'legionnaires'
})

clubs_and_national_players = pd.read_json('../analysis/players_in_national_teams/clubs_and_national_players.json')
clubs_and_national_players = clubs_and_national_players.rename(columns={
    'TeamID': 'team_id',
    'Year': 'year',
    'PlayersInNationalTeam': 'players_in_national_team'
})

team_size_ratio = pd.read_json('../analysis/team_size_ratio/team_size_ratio.json')
team_size_ratio = team_size_ratio.rename(columns={
    'TeamID': 'team_id',
    'Year': 'year',
    'TeamSizeRatio': 'team_size_ratio'
})
 
total_team_cost = pd.read_json('../analysis/total_team_cost/total_team_cost.json')
total_team_cost = total_team_cost.rename(columns={
    'TeamID': 'team_id',
    'Year': 'year',
    'TeamCost': 'team_cost'
})

transfer_balance = pd.read_json('../analysis/transfer_balance/transfer_balance.json')
transfer_balance = transfer_balance.rename(columns={
    'TeamID': 'team_id',
    'Year': 'year',
    'TransferBalance': 'transfer_balance'
})

```sql

teams (
    team_id INT PRIMARY KEY,
    team_name TEXT NOT NULL,
    number_of_cups INT,
    image_link TEXT,
    national_team_id INT REFERENCES national_teams(national_team_id)
);


national_teams (
    national_team_id INT PRIMARY KEY,
    national_team_name TEXT NOT NULL
);

team_yearly_stats (
    team_id INT REFERENCES teams(team_id),
    year INT,
    average_points DECIMAL(3,2),
    average_age DECIMAL(3,1),    
    number_of_titles_this_year INT,
    team_cost INT,
    team_size_ratio DECIMAL(4,2),
    players_in_national_team INT,
    legionnaires INT,
    transfer_balance INT,
    PRIMARY KEY (team_id, year)
);

In [76]:
national_teams = country_info.copy(deep=True)
national_teams = national_teams.drop(['team_ids'], axis=1)
national_teams.head()


,national_team_id,national_team_name
0,3262,Германия
1,3299,Англия
2,3300,Португалия
3,3375,Испания
4,3376,Италия


In [77]:
unique_average_points = average_points.drop(['average_points', 'year'], axis=1)
unique_average_points = unique_average_points.drop_duplicates(subset=['team_id'])
unique_average_points

,team_id,team_name
0,3,Кёльн
11,5,Милан
22,11,Арсенал
33,12,Рома
44,13,Атлетико Мадрид
...,...,...
5536,64534,Струга
5542,64780,Атлетик Эскальд
5549,69752,Хегельманн
5556,80996,Малишево


In [78]:
teams = club_info.merge(unique_average_points, how='outer', on=['team_id', 'team_name'])
teams

,team_id,team_name,number_of_cups,national_team_id,image_link
0,3,Кёльн,13.0,3262.0,https://tmssl.akamaized.net//images/wappen/hea...
1,5,Милан,53.0,3376.0,https://tmssl.akamaized.net//images/wappen/hea...
2,11,Арсенал,48.0,3299.0,https://tmssl.akamaized.net//images/wappen/hea...
3,12,Рома,17.0,3376.0,https://tmssl.akamaized.net//images/wappen/hea...
4,13,Атлетико Мадрид,33.0,3375.0,https://tmssl.akamaized.net//images/wappen/hea...
...,...,...,...,...,...
531,64534,Струга,NaN,NaN,NaN
532,64780,Атлетик Эскальд,2.0,10533.0,https://tmssl.akamaized.net//images/wappen/hea...
533,69752,Хегельманн,0.0,3851.0,https://tmssl.akamaized.net//images/wappen/hea...
534,80996,Малишево,0.0,53982.0,https://tmssl.akamaized.net//images/wappen/hea...


In [79]:
team_yearly_stats = average_age.merge(average_points.drop(['team_name'], axis=1), how='outer', on=['team_id', 'year']) \
    .merge(club_titles, how='outer', on=['team_id', 'year']) \
    .merge(total_team_cost, how='outer', on=['team_id', 'year']) \
    .merge(team_size_ratio, how='outer', on=['team_id', 'year']) \
    .merge(clubs_and_national_players, how='outer', on=['team_id', 'year']) \
    .merge(legionnaires_per_team, how='outer', on=['team_id', 'year']) \
    .merge(transfer_balance, how='outer', on=['team_id', 'year'])
team_yearly_stats

,team_id,year,average_age,average_points,number_of_titles_this_year,team_cost,team_size_ratio,players_in_national_team,legionnaires,transfer_balance
0,3,2014,25.8,1.24,0.0,50750.0,NaN,9,22,-8300.0
1,3,2015,25.4,1.28,0.0,68630.0,0.93,8,17,-1730.0
2,3,2016,25.8,1.49,0.0,116150.0,1.20,5,23,10650.0
3,3,2017,24.9,0.79,2.0,91730.0,1.27,4,28,-32550.0
4,3,2018,25.3,1.83,2.0,83500.0,0.89,3,20,29900.0
...,...,...,...,...,...,...,...,...,...,...
5891,98841,2020,NaN,NaN,0.0,NaN,NaN,0,0,NaN
5892,98841,2021,NaN,2.00,0.0,NaN,NaN,0,3,NaN
5893,98841,2022,NaN,3.00,1.0,NaN,NaN,0,7,NaN
5894,98841,2023,NaN,1.10,2.0,NaN,NaN,1,16,NaN


In [ ]:
from sqlalchemy import Integer, String, Text, DECIMAL

In [81]:
national_teams.to_sql('national_teams', 
    engine, 
    dtype={
        'national_team_id': Integer(),
        'national_team_name': String(100)
    },
    if_exists='append',
    index=False
)

51

In [82]:
teams.to_sql('teams', 
    engine, 
    dtype={
        'team_id': Integer(),
        'team_name': String(100),
        'number_of_cups': Integer(),
        'image_link': Text(),
        'national_team_id': Integer()
    },
    if_exists='append',
    index=False
)

536

In [83]:
team_yearly_stats.to_sql('team_yearly_stats', 
    engine, 
    dtype={
        'team_id': Integer(),
        'year': Integer(),
        'average_points': DECIMAL(3, 2),
        'average_age': DECIMAL(3, 1),
        'number_of_titles_this_year': Integer(),
        'team_cost': Integer(),
        'team_size_ratio': DECIMAL(4, 2),
        'players_in_national_teams': Integer(),
        'legionnaires': Integer(),
        'transfer_balance': Integer()
    },
    if_exists='append',
    index=False
)

896